## 6.7 门控制循环单元(GRU)

> 1. **当时间步较大或较小时,`RNN`容易出现衰减或爆炸**
> 2. **虽然裁剪梯度可以应对梯度爆炸,但无法解决梯度衰减;因此`RNN`较难捕捉时间序列中时间步距离较大的依赖关系**

### 6.7.1 门控循环单元

> **引入了重置门(reset gate)和更新们(update gate),从而修改了`RNN`隐藏状态的计算方式**

#### 6.7.1.1 重置门和更新门

> 1. **门控循环单元中的重置门和更新门的输入均为当前时间步输入$\boldsymbol{X}_{t}$与上一时间步隐藏状态$\boldsymbol{H}_{t-1}$**
> 2. **设隐藏单元个数为$h$,给定时间步$t$的小批量输入$\boldsymbol{X}_{t} \in \mathbb{R}^{n \times d}$(样本数为$n$,输入个数为$d$)和上一时间步隐藏状态$\boldsymbol{H}_{t-1} \in \mathbb{R}^{n \times h}$;重置门$\boldsymbol{R}_{t} \in \mathbb{R}^{n \times h}$和跟新门$\boldsymbol{Z}_{t} \in \mathbb{R}^{n \times h}$计算如下:**
$$\begin{aligned} \boldsymbol{R}_{t} = \sigma(\boldsymbol{X}_{t}\boldsymbol{W}_{xr} + \boldsymbol{H}_{t-1}\boldsymbol{W}_{hr} + \boldsymbol{b_{r}}),\end{aligned} $$
$$\begin{aligned} \boldsymbol{Z}_{t} = \sigma(\boldsymbol{X}_{t}\boldsymbol{W}_{xz} + \boldsymbol{H}_{t-1}\boldsymbol{W}_{hz} + \boldsymbol{b_{z}}),\end{aligned} $$
**其中$\boldsymbol{W}_{xr}, \boldsymbol{W}_{xz} \in \mathbb{R}^{d \times h}$和$\boldsymbol{W}_{hr}, \boldsymbol{W}_{hz} \in \mathbb{R}^{h \times h}$是权重参数,$\boldsymbol{b}_r, \boldsymbol{b}_z \in \mathbb{R}^{1 \times h}$;使用sigmod函数的值域为$[0, 1]$,所以重置门$\boldsymbol{R}_{t}$和更新门$\boldsymbol{Z}_{t}$的值域也是$[0, 1]$**

![门控循环单元中重置门和更新门的计算](img/6.7_gru_1.svg)

#### 6.7.1.2 候选隐藏状态

> 1. **门控循环单元将计算`候选隐藏状态`来辅助稍后的隐藏状态计算**
> 2. **当前时间步重置门的输出与上一时间步隐藏状态做按元素乘法(符号位$\odot$)**
>      - **如果重置门元素值接近0,意味着隐藏状态元素为0,则丢弃上一时间步的隐藏状态**
>      - **如果接近1,则保留上一时间步的隐藏状态**
>      - **将乘法结果与当前实践部的输入连接,通过激活函数$tanh$的全连接层计算出候选隐藏状态,值域$[-1, 1]$**
> 3. **时间步$t$的候选隐藏状态$\tilde{\boldsymbol{H}}_t \in \mathbb{R}^{n \times h}$的计算为:**
$$\tilde{\boldsymbol{H}_{t}} = \text{tanh}(\boldsymbol{X}_{t}\boldsymbol{W}_{xh} + \left(\boldsymbol{R}_{t} \odot \boldsymbol{H}_{t-1}\right) \boldsymbol{W}_{hh} + \boldsymbol{b}_{h}),$$

> 4. **重置门控制了上一时间步的隐藏状态如何流入当前时间步的候选隐藏状态;而上一时间步的隐藏状态可能包含了时间序列截至上一时间步的全部历史信息**
> 5. **因此重置门可以用来丢弃与预测无关的历史信息**


![门控循环单元中候选隐藏状态的计算](img/6.7_gru_2.svg)

#### 6.7.1.3 隐藏状态

> 1. **时间步$t$的隐藏状态$\boldsymbol{H}_t \in \mathbb{R}^{n \times h}$的计算使用当前时间步的更新门$\boldsymbol{Z}_{t}$来对上一时间步的隐藏状态$\boldsymbol{H}_{t-1}$和当前时间步的候选隐藏状态$\tilde{\boldsymbol{H}}_{t}$做组合**
$$\boldsymbol{H} = \boldsymbol{Z}_{t} \odot \boldsymbol{H}_{t-1} + \left(1 - \boldsymbol{Z}_{t}  \right) \odot \tilde{\boldsymbol{H}_{t}}$$

> 2. **更新门可以控制隐藏状态应该如何被包含当前时间步信息的候选隐藏状态所更新**
> 3. **假设更新门在时间步$t'$到$t$（$t' < t$）之间一直近似1,那么$t'$到$t$之间的信息几乎`没有流入`时间步$t$的隐藏状态$\boldsymbol{H}_{t}$(因为上面公式的后半部分  $\left(1 - \boldsymbol{Z}_{t}  \right) \odot \tilde{\boldsymbol{H}_{t}}$  几乎为0)**
> 4. **这样可以看错较早时刻的隐藏状态$\boldsymbol{H}_{t'-1}$一直通过时间保存到当前**
> 5. **这个设计可以应对循环神经网络中的梯度衰减问题,并更好地捕捉时间序列中时间步距离较大的依赖关系**
> 6. **对门控循环单元总结**
>     - **重置门有助于捕捉时间序列里短期的依赖关系**
>     - **更新门有助于捕捉时间序列里长期的依赖关系**

![门控循环单元中隐藏状态的计算](img/6.7_gru_3.svg)

### 6.7.2 读取数据集

In [2]:
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..")
import d2lzh_pytorch.utils as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

### 6.7.3 从零开始实现

#### 6.7.3.1 初始化模型参数

In [4]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), dtype=torch.float32, device=device)
        return torch.nn.Parameter(ts, requires_grad=True)
    
    def _three():
        return (_one((num_inputs, num_hiddens)),
               _one((num_hiddens, num_hiddens)),
               torch.nn.Parameter(torch.zeros(num_hiddens, dtype=torch.float32, device=device)))
    
    W_xz, W_hz, b_z = _three()  # 更新门参数
    W_xr, W_hr, b_r = _three()  # 重置门参数
    W_xh, W_hh, b_h = _three()  # 候选隐藏状态参数
    
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q])

#### 6.7.3.2 定义模型

In [5]:
# 返回形状(批量大小, 隐藏单元个数)的值为0的Tensor组成的元祖
def init_gru_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

In [6]:
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        Z = torch.sigmoid(torch.matmul(X, W_xz) + torch.matmul(H, W_hz) + b_z)
        R = torch.sigmoid(torch.matmul(X, W_xr) + torch.matmul(H, W_hr) + b_r)
        H_tilda = torch.tanh(torch.matmul(X, W_xh) + R * torch.matmul(H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, )

#### 6.7.3.3 训练模型并创造歌词

In [7]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [8]:
d2l.train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)

epoch 40, perplexity 150.849720, time 1.21 sec
 - 分开 我想你的让我不想想想想你你的爱爱女人 爱爱我的让我爱爱女女 爱爱我的可爱爱女女 坏坏的让我爱爱女人
 - 不分开 我想你的让我不想想想想你你的爱爱女人 爱爱我的让我爱爱女女 爱爱我的可爱爱女女 坏坏的让我爱爱女人
epoch 80, perplexity 32.048513, time 1.10 sec
 - 分开 我想要这样 我有一定个人 我想要你的微笑 让我想这样 我不要再想 我不要再想 我不要再想 我不要再
 - 不分开 我想要你 我不要再想 我不要再想 我不能再想 我不要再想 我不要再想 我不要再想 我不要再想 我不
epoch 120, perplexity 4.776606, time 1.13 sec
 - 分开 我想要这样牵很久 别想躲 说你眼睛看着我 别发抖 快给我抬起头 有话去对医药箱说 别怪我 别怪我 
 - 不分开 爱情不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我
epoch 160, perplexity 1.428972, time 1.71 sec
 - 分开 我想要你的微笑每天都能看到  我知道这里很美但家乡的你更美原来我只想要你 陪我去吃汉堡  说穿了其
 - 不分开 不要再这样打我妈妈 难道你手不会痛吗 其实我回家就想要阻止一切 让家庭回到过去甜甜 温馨的欢乐香味


### 6.7.4 简洁实现

In [10]:
lr = 1e-2 
gru_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens)
model = d2l.RNNModel(gru_layer, vocab_size).to(device)
d2l.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.016311, time 0.84 sec
 - 分开始的路上 时间变好慢 老街坊 小弄堂 是属于那年代白墙黑瓦的淡淡的忧伤 消失的 旧时光 一九四三 回
 - 不分开始的担忧 唱着歌 一直走 我想就这样牵着你的手不放开 爱可不可以简简单单没有伤害 你 靠着我的肩膀 
epoch 80, perplexity 1.011174, time 0.82 sec
 - 分开 我不是你不想活 说你怎么面对我 甩开球我满腔的怒火 我想揍你已经很久 别想躲 说你眼睛看着我 别发
 - 不分开始 担心今天的你过得好不好 整个画面是你 想你想的睡不著 嘴嘟嘟那可爱的模样 还有在你身上香香的味道
epoch 120, perplexity 1.010766, time 0.82 sec
 - 分开 爱能不能够永远单纯没有悲哀 我 想带你骑单车 我 想和你看棒球 想这样没担忧 唱着歌 一直走 我想
 - 不分开  其实我早已经猜透看透不想多说 只是我怕眼泪撑不住 不懂 你的黑色幽默 想通 却又再考倒我 说散 
epoch 160, perplexity 1.019968, time 1.03 sec
 - 分开 爱能不能够永远单纯没有悲哀 我 想带你骑单车 我 想和你看棒球 想这样没担忧 唱着歌 一直走 我想
 - 不分开球 印地安的传说 还真是 瞎透了 什么都有 沙漠之中怎么会有泥鳅 话说完飞过一只海鸥 大峡谷的风呼啸
